In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from collections import Counter
import datetime
import math
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import multi_gpu_model
from livelossplot.inputs.tf_keras import PlotLossesCallback

In [ ]:
column_names = ['path', 'make', 'model', 'year_of_production']
raw_dataset = pd.read_csv('./cars.csv', names = column_names, sep=';',
                      dtype={"path": str, "make": str, 'model': str, 'year_of_production': str})

raw_dataset = raw_dataset[raw_dataset.path != 'path']
raw_dataset = raw_dataset[raw_dataset.year_of_production != 'ints']

train_paths = raw_dataset['path'].values.tolist()
make = raw_dataset['make'].values.tolist()

BATCH_SIZE = 16
IMG_WIDTH = 220
IMG_HEIGHT = 220

In [ ]:
def prepare_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_AREA)
    img = img/255.0
    return img

In [ ]:
os.chdir('Car_Dataset')

In [ ]:
CLASS_NAMES_MAKE = np.array(np.unique([name for name in make]))

In [ ]:
def get_label_make(labels):
    make = tf.strings.split(labels, '_')[0]
    return make == CLASS_NAMES_MAKE

In [ ]:
images = []
label_make = []
for path in train_paths:
    images.append(prepare_image(path))
    label_make.append(path)
images = np.array(images)
label_make = np.array(label_make)

In [ ]:
image_input = keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3), name='image')
x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)

output_make = layers.Dense(len(CLASS_NAMES_MAKE), activation='softmax', name='output_make')(x)

model = keras.Model(inputs=image_input, outputs=output_make, name='cars_model')

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
plot_losses = PlotLossesCallback()
model.fit(images, makes,
          epochs = 20,
          batch_size = BATCH_SIZE,
          callbacks=[plot_losses])